### glacier elevatin change statistics based on data by Huggonet et al.(2021) .


In [1]:
import rasterio as rio
import numpy as np
import geopandas as gpd
from rasterio.enums import Resampling
from rasterstats import zonal_stats


In [2]:
path_dhdt_gla = 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/dhdt_mosaic.tif'
path_hma_gtng = 'data/hma-extent/HMA/hma_gtng_202307_subregions.gpkg'
path_gla_larger1 = 'data/glacier-extent/rgi70/hma_rgi70_larger1km2.gpkg'


In [3]:
## read hma and glacier extent
hma_gtng_gdf = gpd.read_file(path_hma_gtng)
gla_larger1_gdf = gpd.read_file(path_gla_larger1)


In [4]:
### read dhdt data
with rio.open(path_dhdt_gla) as src:
    arr_down = src.read(
        1,  # Read the first band
        out_shape=(src.height // 3, src.width // 3),  # Downsampling
        resampling=Resampling.bilinear
        )
    src_bounds = src.bounds  # Get the bounds of the source raster
arr_down[arr_down == src.nodata] = np.nan  # Set nodata values to NaN
dhdt_extent = [src_bounds.left, src_bounds.right, src_bounds.bottom, src_bounds.top]


In [5]:
stats = zonal_stats(
    vectors=gla_larger1_gdf,         
    raster=path_dhdt_gla,   #  path of the raster file
    stats=['mean', 'median', 'std', 'count'],  # statistics to compute
    geojson_out=True,  # output as GeoJSON
    nodata=src.nodata)


In [6]:
## convert to GeoDataFrame
gla_stats_gdf = gpd.GeoDataFrame.from_features(stats, crs=gla_larger1_gdf.crs)
gla_stats_gdf = gla_stats_gdf[['rgi_id', 'area_km2', 'mean', 'median', 'std', 'count', 'geometry']]
gla_stats_gdf.head(5)
# gla_stats_gdf.to_file('data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/gla_rgi70_dhdt_stats.gpkg', driver='GPKG')


,rgi_id,area_km2,mean,median,std,count,geometry
0,RGI2000-v7.0-G-13-00248,2.317426,-0.056183,-0.025720,0.463052,178,"MULTIPOLYGON Z (((68.11814 39.07499 0, 68.1177..."
1,RGI2000-v7.0-G-13-00333,1.566492,0.120341,0.017896,0.340954,142,"MULTIPOLYGON Z (((68.28872 39.1812 0, 68.28813..."
2,RGI2000-v7.0-G-13-00337,1.248292,-0.087538,0.008667,0.557182,63,"MULTIPOLYGON Z (((68.29033 39.1667 0, 68.29011..."
3,RGI2000-v7.0-G-13-00342,1.367711,-0.120291,-0.126801,0.287575,106,"MULTIPOLYGON Z (((68.31505 39.15983 0, 68.3151..."
4,RGI2000-v7.0-G-13-00358,3.060479,0.000685,-0.000928,0.312978,281,"MULTIPOLYGON Z (((68.32432 39.15742 0, 68.3243..."
